In [1]:
! pip install python-telegram-bot

     -------------------------------------- 513.4/513.4 kB 3.2 MB/s eta 0:00:00
     -------------------------------------- 160.2/160.2 kB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 58.9/58.9 kB ? eta 0:00:00
     -------------------------------------- 339.5/339.5 kB 5.3 MB/s eta 0:00:00


In [2]:
! pip install nltk

     ---------------------------------------- 1.5/1.5 MB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB 5.8 MB/s eta 0:00:00
     -------------------------------------- 307.0/307.0 kB 6.3 MB/s eta 0:00:00
     -------------------------------------- 262.1/262.1 kB 2.3 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 kB 4.3 MB/s eta 0:00:00


In [3]:
! pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.4/7.4 MB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 36.9/36.9 MB 5.4 MB/s eta 0:00:00
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'


In [1]:
import json

import re
import nltk
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression

import pickle
import telegram
import os

<div class="alert alert-block alert-info">
<b> Файловая часть </b>
</div>

Создание структуры из файлов с наборами слов

In [6]:
name_files = ['geochemistry.txt', 'lithological description + core.txt',
              'name of well, profile, area.txt',
              'sedimentary environments.txt', 'paleontology.txt',
              'stratigraphy.txt', 'subjects of the Russian Federation.txt', 'tectonics.txt']

In [7]:
#for name in name_files
name_words_category = []
for numb in range(len(name_files)):
    with open (name_files[numb], 'r') as file: 
        l = []
        for i in file:
            i = i.split()
            l += i
    name_words_category += [l]
name_words_category

[['органический',
  'битуминозный',
  'сероводорода',
  'детрита',
  'органоген',
  'углефицированного',
  'H2S'],
 ['стилолитоподобными',
  'желвакообразными',
  'ангидритово-доломитовом',
  'светлого',
  'дисперсный',
  'цемент',
  'Доломит',
  'оолитовые',
  'биоморфно-детритовый',
  'кристаллическими',
  'межзернового',
  'крупно-грубозернистых',
  'разнокристаллический',
  'изломе',
  'пленочно-порового',
  'микротрещины',
  'среднезернистого',
  'темно-серого',
  'хрупкого',
  'коричневато-серый',
  'тонко-мелкозернистый',
  'кремово-серую',
  'параллельная',
  'грязно-серого',
  'детрита',
  'Алевропесчаник',
  'перекристаллизованные',
  'зеленовато-серым',
  'Мергель',
  'доломитовый',
  'мелкие',
  'полупрозрачный',
  'чешуек',
  'серо-бурого',
  'реликтовой',
  'окраске',
  'окатаны',
  'грязно-белого',
  'тонкодисперсный',
  'слабослюдистый',
  'розовыми',
  'зонтальной',
  'блеском',
  'кавернозный',
  'глинисто-карбонатном',
  'зернистых',
  'алевро-песчаным',
  'матриксе'

In [8]:
BOT_CONF_FILES = {
     "categories": {
         "geochemistry": {
             "examples": name_words_category[0],
             "response": ["Геохимия"],
         },
         "lithology": {
             "examples": name_words_category[1],
             "responses": ["Литология, возможно керн"]
         },
         "wells": {
             "examples": name_words_category[2],
             "responses": ["Название скважины, профиля, площади"],      
         },
         "sedimentation": {
             "examples": name_words_category[3],
             "responses": ["Обстановки осадконакопления"],      
         },
         "paleontology": {
             "examples": name_words_category[4],
             "responses": ["Палеонтология"],      
         },
         "stratigraphy": {
             "examples": name_words_category[5],
             "responses": ["Стратиграфия"],      
         },
         "subjectsRUS": {
             "examples": name_words_category[6],
             "responses": ["Субъекты РФ"],      
         },
         "tectonics": {
             "examples": name_words_category[7],
             "responses": ["Тектоника"],      
         },
     },
     "failure_phrases": ["К сожалению, я ничего не понял"]
 }

In [9]:
with open("config_for_files.json", "w") as write_file:
    json.dump(BOT_CONF_FILES, write_file)

<div class="alert alert-block alert-success">
<b> Функции </b>
</div>

In [10]:
config_file = open("config_for_files.json", "r")
BOT_CONF_FILES = json.load(config_file)

In [11]:
def normalize(text): 
    """
    text - str
    Function removes punctuation marks and converts text to lowercase.

    Return text (str).
    """
    text = text.lower() 
    punctuation = r"[^\w\s]"
    return re.sub(punctuation, "", text) 

def isMatching(text1, text2):
    """
    text1 - str
    text2 - str

    Function finds same texts.
    Return match percentage (int).
    """ 
    text1 = normalize(text1)
    text2 = normalize(text2)
    distance = nltk.edit_distance(text1, text2) 
    average_length = (len(text1) + len(text2)) / 2 
    return distance / average_length < 0.4

def getIntent(text): 
    """
    text - str

    Function looks for the text category in BOT_CONFIG.
    Return category answers if find text category (str).
    """
    all_intents = BOT_CONF_FILES["categories"]
    for name, data in all_intents.items(): 
        for example in data["examples"]: 
            if isMatching(text, example):
                return name

def getAnswer(intent):
    """
    intent - str

    Function return one random answer from answers list (str).
    """
    responses = BOT_CONF_FILES["categories"][intent]["responses"]
    return random.choice(responses)

In [12]:
def bot_files(text):
    """
    text - str

    Function tries to define an intent. If the intent is not found, then the model predicts the answer, otherwise the phrase from 'failure_phrases'.
    Return answer (str).
    """  
    list_categories = []
    file = open(text, 'r', encoding="UTF-8")
    for string in file:

        intent = getIntent(string)
        
        if not intent:
            test = vectorizer_files.transform([string])
            intent = model_files.predict(test)[0] 

        print("Intent =", intent)

        if intent:
            list_categories += [getAnswer(intent)]
        
    if len(list_categories) > 0:
        return list_categories
    else:
        return BOT_CONF_FILES['failure_phrases']

<div class="alert alert-block alert-success">
<b> Обучение </b>
</div>

In [14]:
# texts
X = []
# category
y = []
for name, data in BOT_CONF_FILES["categories"].items():
    for example in data['examples']:
        X.append(example) # append texts in X
        y.append(name) # append categories in y

In [15]:
vectorizer_files = TfidfVectorizer()
vectorizer_files.fit(X) 

TfidfVectorizer()

In [16]:
X_vectorized = vectorizer_files.transform(X) 

In [17]:
model_files = LogisticRegression(solver='saga')
model_files.fit(X_vectorized, y)

LogisticRegression(solver='saga')

In [18]:
test = vectorizer_files.transform(["Шельф"])
model_files.predict(test)

array(['wells'], dtype='<U13')

In [19]:
model_files.score(X_vectorized, y)

0.9608545647558386

In [20]:
f = open("bot_model_for_files.bin", "wb")
pickle.dump(model_files, f)

<div class="alert alert-block alert-success">
<b> Загрузка </b>
</div>

In [21]:
f = open("bot_model_for_files.bin", "rb")
loaded_model = pickle.load(f)

In [22]:
loaded_model

LogisticRegression(solver='saga')

In [24]:
bot_files('test_file.txt')

Intent = stratigraphy
Intent = wells
Intent = paleontology
Intent = wells


['Стратиграфия',
 'Название скважины, профиля, площади',
 'Палеонтология',
 'Название скважины, профиля, площади']

<div class="alert alert-block alert-info">
<b> Запуск бота </b>
</div>

In [25]:
# BotFather
BOT_KEY = 'str' #enter your bot key

In [27]:
from telegram import Update
from telegram.ext import Updater, CallbackContext, MessageHandler, Filters


def file_handler(update: Update, context: CallbackContext):
    update.message.reply_text('Читаю файл')
    file = context.bot.getFile(update.message.document.file_id)
    file.download(update.message.document.file_name)
    
    doc = update.message.document
    filename = doc.file_name
    
    print(f"Message: {doc}")
    reply = bot_files(filename)
    print(str(reply))
    for i in reply:
        update.message.reply_text(str(i))

updater = Updater(BOT_KEY)

updater.dispatcher.add_handler(MessageHandler(Filters.document, file_handler))

updater.start_polling()
updater.idle()

Message: {'file_id': 'BQACAgIAAxkBAAIBnGLBpx0qVuK-dVnHo9wBmrQpngfOAAIgGQACrv4JSqcVFi_hpNFqKQQ', 'mime_type': 'text/plain', 'file_unique_id': 'AgADIBkAAq7-CUo', 'file_name': 'test.txt', 'file_size': 119}
Intent = wells
Intent = subjectsRUS
Intent = lithology
Intent = sedimentation
Intent = wells
Intent = paleontology
['Название скважины, профиля, площади', 'Субъекты РФ', 'Литология, возможно керн', 'Обстановки осадконакопления', 'Название скважины, профиля, площади', 'Палеонтология']
